In [1]:
import os
import sys

import numpy as np
import pandas as pd

import seaborn as sns

# add gaugi by hand
#sys.path.append('../../.__python__/')
from Gaugi import load as gload

ModuleNotFoundError: No module named 'Gaugi'

In [2]:
!jupyter-troubleshoot

/bin/sh: 1: jupyter-troubleshoot: not found


In [5]:
os.environ

environ{'HOSTNAME': 'micael-notebook',
        'LD_LIBRARY_PATH': '/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64',
        'HOME': '/root',
        'CUDA_VERSION': '10.1.243',
        'NVIDIA_REQUIRE_CUDA': 'cuda>=10.1 brand=tesla,driver>=384,driver<385 brand=tesla,driver>=396,driver<397 brand=tesla,driver>=410,driver<411',
        'NVIDIA_DRIVER_CAPABILITIES': 'compute,utility',
        'PATH': '/root/.vscode-server/bin/ae08d5460b5a45169385ff3fd44208f431992451/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
        'VSCODE_AGENT_FOLDER': '/root/.vscode-server',
        'CUDA_PKG_VERSION': '10-1=10.1.243-1',
        'LANG': 'C.UTF-8',
        'SHELL': '/bin/bash',
        'PWD': '/root/.vscode-server/bin/ae08d5460b5a45169385ff3fd44208f431992451',
        'NVIDIA_VISIBLE_DEVICES': 'all',
        'VSCODE_INJECT_NODE_MODULE_LOOKUP_PATH': '/root/.vscode-server/bin/ae08d5460